In [2]:
from Bio import SeqIO, motifs
from Bio.Seq import Seq
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from collections import Counter
import shutil
import os
import csv
from io import StringIO
import random
import re
from Bio.SeqRecord import SeqRecord
import torch
from pycaret.classification import *
import json
import seaborn as sns

from sklearn.model_selection import train_test_split

In [5]:
training1 = pd.read_csv("Classification_1/training.csv", index_col=None)

In [6]:
testing1 = pd.read_csv("Classification_1/testing.csv", index_col=None)
testing1.index = pd.RangeIndex(start=224634, stop=280792)

In [7]:
training1

,0,1,2,3,4,5,6,7,8,9,...,1425,1426,1427,1428,1429,1430,1431,1432,1433,Label
0,-0.002948,-0.002335,-0.015625,-0.022707,-0.024158,-0.002981,-0.011018,-0.024943,-0.031998,-0.017779,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
1,-0.014518,-0.003701,-0.020209,-0.018271,-0.029401,-0.007733,-0.013668,-0.026010,-0.027676,-0.010289,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
2,-0.006594,-0.011881,-0.021094,-0.018286,-0.019621,-0.009765,-0.005873,-0.018330,-0.020545,-0.023735,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
3,-0.007294,-0.006613,-0.021188,-0.023279,-0.022387,-0.003038,-0.006053,-0.023679,-0.030497,-0.020677,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
4,-0.002904,-0.015823,-0.019966,-0.021471,-0.020766,-0.001826,-0.007184,-0.021215,-0.026025,-0.020393,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224629,-0.015241,-0.010874,-0.019784,-0.020911,-0.014876,-0.008531,-0.001834,-0.020736,-0.024874,-0.020887,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
224630,-0.004369,-0.012980,-0.021153,-0.019490,-0.011709,0.001926,-0.006773,-0.025501,-0.028967,-0.026096,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
224631,-0.014158,-0.000782,-0.014001,-0.017175,-0.029909,-0.004006,-0.015007,-0.025682,-0.029045,-0.018410,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
224632,-0.012410,-0.005594,-0.010781,-0.018431,-0.018036,-0.000381,-0.007243,-0.022568,-0.027034,-0.018358,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0


In [8]:
testing1

,0,1,2,3,4,5,6,7,8,9,...,1425,1426,1427,1428,1429,1430,1431,1432,1433,Label
224634,-0.019207,-0.004524,-0.018542,-0.009269,-0.029839,-0.004846,-0.001079,-0.023926,-0.016897,-0.014039,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
224635,-0.019207,-0.004524,-0.018542,-0.009269,-0.029839,-0.004846,-0.001079,-0.023926,-0.016897,-0.014039,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
224636,-0.019207,-0.004524,-0.018542,-0.009269,-0.029839,-0.004846,-0.001079,-0.023926,-0.016897,-0.014039,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
224637,-0.019207,-0.004524,-0.018542,-0.009269,-0.029839,-0.004846,-0.001079,-0.023926,-0.016897,-0.014039,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
224638,-0.019207,-0.004524,-0.018542,-0.009269,-0.029839,-0.004846,-0.001079,-0.023926,-0.016897,-0.014039,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280787,-0.009396,-0.015384,-0.016061,-0.017767,-0.016071,-0.009802,-0.012459,-0.023288,-0.019165,-0.014492,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
280788,-0.009396,-0.015384,-0.016061,-0.017767,-0.016071,-0.009802,-0.012459,-0.023288,-0.019165,-0.014492,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
280789,-0.009396,-0.015384,-0.016061,-0.017767,-0.016071,-0.009802,-0.012459,-0.023288,-0.019165,-0.014492,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
280790,-0.009396,-0.015384,-0.016061,-0.017767,-0.016071,-0.009802,-0.012459,-0.023288,-0.019165,-0.014492,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0


In [9]:
from pycaret.classification import *
setup(data=training1, target='Label', test_data=testing1)

,Description,Value
0,Session id,6390
1,Target,Label
2,Target type,Binary
3,Original data shape,"(280792, 1435)"
4,Transformed data shape,"(280792, 1435)"
5,Transformed train set shape,"(224634, 1435)"
6,Transformed test set shape,"(56158, 1435)"
7,Numeric features,1434
8,Preprocess,True
9,Imputation type,simple


In [ ]:
best = compare_models()
save_model(best, 'Classification_1/best')

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,18:36:19
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Logistic Regression


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

In [ ]:
svm_model = compare_models(whitelist=['svm'])
save_model(svm_model, 'Classification_1/svm')

In [3]:
training2 = pd.read_csv("Classification_2/training.csv", index_col=None)
testing2 = pd.read_csv("Classification_2/testing.csv", index_col=None)

In [6]:
testing2.index = pd.RangeIndex(start=224630, stop=280792)

In [4]:
training2

,0,1,2,3,4,5,6,7,8,9,...,1425,1426,1427,1428,1429,1430,1431,1432,1433,Label
0,-0.014312,-0.006958,-0.017485,-0.020401,-0.020887,0.002440,-0.009376,-0.023673,-0.031859,-0.023659,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
1,-0.014312,-0.006958,-0.017485,-0.020401,-0.020887,0.002440,-0.009376,-0.023673,-0.031859,-0.023659,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1
2,-0.014312,-0.006958,-0.017485,-0.020401,-0.020887,0.002440,-0.009376,-0.023673,-0.031859,-0.023659,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
3,-0.014312,-0.006958,-0.017485,-0.020401,-0.020887,0.002440,-0.009376,-0.023673,-0.031859,-0.023659,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
4,-0.014312,-0.006958,-0.017485,-0.020401,-0.020887,0.002440,-0.009376,-0.023673,-0.031859,-0.023659,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224625,-0.007129,-0.019132,-0.020710,-0.028551,-0.012825,-0.007645,-0.006598,-0.020692,-0.025174,-0.022122,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0
224626,-0.007129,-0.019132,-0.020710,-0.028551,-0.012825,-0.007645,-0.006598,-0.020692,-0.025174,-0.022122,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0
224627,-0.007129,-0.019132,-0.020710,-0.028551,-0.012825,-0.007645,-0.006598,-0.020692,-0.025174,-0.022122,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0
224628,-0.007129,-0.019132,-0.020710,-0.028551,-0.012825,-0.007645,-0.006598,-0.020692,-0.025174,-0.022122,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0


In [7]:
testing2

,0,1,2,3,4,5,6,7,8,9,...,1425,1426,1427,1428,1429,1430,1431,1432,1433,Label
224630,-0.014312,-0.006958,-0.017485,-0.020401,-0.020887,0.002440,-0.009376,-0.023673,-0.031859,-0.023659,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1
224631,-0.014312,-0.006958,-0.017485,-0.020401,-0.020887,0.002440,-0.009376,-0.023673,-0.031859,-0.023659,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
224632,-0.014312,-0.006958,-0.017485,-0.020401,-0.020887,0.002440,-0.009376,-0.023673,-0.031859,-0.023659,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
224633,-0.014312,-0.006958,-0.017485,-0.020401,-0.020887,0.002440,-0.009376,-0.023673,-0.031859,-0.023659,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
224634,-0.014312,-0.006958,-0.017485,-0.020401,-0.020887,0.002440,-0.009376,-0.023673,-0.031859,-0.023659,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280787,-0.007129,-0.019132,-0.020710,-0.028551,-0.012825,-0.007645,-0.006598,-0.020692,-0.025174,-0.022122,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0
280788,-0.007129,-0.019132,-0.020710,-0.028551,-0.012825,-0.007645,-0.006598,-0.020692,-0.025174,-0.022122,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0
280789,-0.007129,-0.019132,-0.020710,-0.028551,-0.012825,-0.007645,-0.006598,-0.020692,-0.025174,-0.022122,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0
280790,-0.007129,-0.019132,-0.020710,-0.028551,-0.012825,-0.007645,-0.006598,-0.020692,-0.025174,-0.022122,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0


In [8]:
training3 = pd.read_csv("Classification_3/training.csv", index_col=None)
testing3 = pd.read_csv("Classification_3/testing.csv", index_col=None)

In [9]:
training3

,0,1,2,3,4,5,6,7,8,9,...,1425,1426,1427,1428,1429,1430,1431,1432,1433,Label
0,-0.027982,-0.002226,-0.019630,-0.014697,-0.026953,-0.004544,-0.001856,-0.028293,-0.020626,-0.017536,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
1,-0.027982,-0.002226,-0.019630,-0.014697,-0.026953,-0.004544,-0.001856,-0.028293,-0.020626,-0.017536,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
2,-0.027982,-0.002226,-0.019630,-0.014697,-0.026953,-0.004544,-0.001856,-0.028293,-0.020626,-0.017536,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
3,-0.027982,-0.002226,-0.019630,-0.014697,-0.026953,-0.004544,-0.001856,-0.028293,-0.020626,-0.017536,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
4,-0.027982,-0.002226,-0.019630,-0.014697,-0.026953,-0.004544,-0.001856,-0.028293,-0.020626,-0.017536,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195985,-0.009396,-0.015384,-0.016061,-0.017767,-0.016071,-0.009802,-0.012459,-0.023288,-0.019165,-0.014492,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
195986,-0.009396,-0.015384,-0.016061,-0.017767,-0.016071,-0.009802,-0.012459,-0.023288,-0.019165,-0.014492,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
195987,-0.009396,-0.015384,-0.016061,-0.017767,-0.016071,-0.009802,-0.012459,-0.023288,-0.019165,-0.014492,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
195988,-0.009396,-0.015384,-0.016061,-0.017767,-0.016071,-0.009802,-0.012459,-0.023288,-0.019165,-0.014492,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0


In [12]:
testing3

,0,1,2,3,4,5,6,7,8,9,...,1425,1426,1427,1428,1429,1430,1431,1432,1433,Label
195990,-0.019207,-0.004524,-0.018542,-0.009269,-0.029839,-0.004846,-0.001079,-0.023926,-0.016897,-0.014039,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
195991,-0.019207,-0.004524,-0.018542,-0.009269,-0.029839,-0.004846,-0.001079,-0.023926,-0.016897,-0.014039,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
195992,-0.019207,-0.004524,-0.018542,-0.009269,-0.029839,-0.004846,-0.001079,-0.023926,-0.016897,-0.014039,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
195993,-0.019207,-0.004524,-0.018542,-0.009269,-0.029839,-0.004846,-0.001079,-0.023926,-0.016897,-0.014039,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
195994,-0.019207,-0.004524,-0.018542,-0.009269,-0.029839,-0.004846,-0.001079,-0.023926,-0.016897,-0.014039,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280787,-0.019380,-0.008876,-0.017869,-0.025646,-0.017708,-0.002925,-0.016797,-0.026565,-0.036883,-0.012422,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
280788,-0.019380,-0.008876,-0.017869,-0.025646,-0.017708,-0.002925,-0.016797,-0.026565,-0.036883,-0.012422,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0
280789,-0.019380,-0.008876,-0.017869,-0.025646,-0.017708,-0.002925,-0.016797,-0.026565,-0.036883,-0.012422,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0
280790,-0.019380,-0.008876,-0.017869,-0.025646,-0.017708,-0.002925,-0.016797,-0.026565,-0.036883,-0.012422,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0


In [11]:
testing3.index = pd.RangeIndex(start=195990, stop=280792)